In [2]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

# turn off GUI, use application correlating to Filename
stk_object = STK_Simulation(False,Filename)

In [3]:
# Setting the time format in STK to UTCG (Coordinated Universal Time in Gregorian calendar)
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")

# Convert the start time of the current STK scenario into a datetime object using the 'time_convert' function
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)

# Define a duration of 1 day using the timedelta function
# This creates a time period of 1 day, which will be added to the scenario start time
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)

# Calculate the stop time by adding the duration to the start time
# The result is formatted as a string in the format "DD MMM YYYY HH:MM:SS.SSSSSS"
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")

# Set the stop time of the current STK scenario to the calculated stop time
stk_object.root.CurrentScenario.StopTime=stop_time

# Set the time format back to EpSec (Ephemeris Seconds, which is time elapsed from a reference epoch)
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# Define the parameters for the optimizer
# 'n_pop' is the number of individuals in the population (10 in this case)
# 'n_gen' is the number of generations (iterations of the optimization process, 10 in this case)
# 'n_sats' is the number of satellites (5 satellites to be optimized)
n_pop = 10
n_gen = 10
n_sats = 5

# Initialize an optimizer object with the STK object, population size, number of generations, and number of satellites
# 'weights' is a tuple that defines the weights for the optimization objective function (7, -5, -1 in this case)
# These weights likely correspond to the importance of different objectives being optimized
opt = Optimizer(stk_object,n_pop,n_gen,n_sats,weights=(7,-5,-1))

In [4]:
# Load target data into the STK object
# The 'Target_Loader' method is used to load information from the specified targets file.
# The file "../../Input_Files/Targets_File.txt" contains data about targets that will be used in the scenario.
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")

# Load satellite data into the STK object
# The 'Satellite_Loader' method is used to load information from the specified satellites file.
# The file "../../Input_Files/Satellites_File.txt" contains data about the satellites involved in the scenario.
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [12]:
stk_object.dt = 30
stk_object.Interpolate=True

# Run optimization if read=True the 1st generation takes the input file as one of the initial population
hof,percent,std,time = opt.run(read=True)

print("Hall of Fame:")
for entry in hof:
    print(entry,"->",entry.fitness)

# Get best individual from optimization and load them into ../../Input_Files/Satellite_File.txt

best_stats = opt.cost_function(hof[0])

-- Generation 10 --
         avg       std        min        max
0  70.935185  0.250000  70.185185  71.018519
1   4.268459  0.107556   3.945792   4.304311
2  19.299164  0.060538  19.117548  19.319343
Hall of Fame:
[612, 67, 168, 5] -> (np.float64(71.0493827160494), np.float64(4.1724924978077), np.float64(19.27445727748917))
[612, 67, 158, 5] -> (np.float64(71.0185185185185), np.float64(4.304310851544242), np.float64(19.319343008105278))
[611, 67, 158, 5] -> (np.float64(70.77160493827162), np.float64(4.13176973094907), np.float64(19.315100322564167))
[611, 67, 118, 5] -> (np.float64(70.74074074074073), np.float64(4.115020570987396), np.float64(19.49460437667583))
[611, 67, 168, 5] -> (np.float64(70.61728395061729), np.float64(4.217906248359632), np.float64(19.27022422864361))


In [6]:
stk_object.dt = 2.5
stk_object.Generate_Holding_Data()

- Computing_AER Data/Default [==========] 20/20 [100%] in 1.0s (19.68/s)        
- Computing_Sat Angles Data [==========] 20/20 [100%] in 0.8s (26.01/s)         
- Compiling_Data [==========] 10/10 [100%] in 0.1s (95.87/s)                    
- Creating_Hash_Map [==========] 3240/3240 [100%] in 0.2s (19856.28/s)          


0

In [7]:
stk_object.Plan(slew_rate=1,cone_angle=20)

- Planning [==========] 3240/3240 [100%] in 1.3s (2951.67/s)                    


0

In [8]:
# Initialize an empty dictionary to store the comparison data
data_comparison = {}


# Calculate the percentage of unique unplanned bin numbers for each target
# 'stk_object.Holding_Data' contains unplanned data, filtered for each target ('Target' == tar_num)
# For each target, count the number of unique 'Bin Number' values, divide by 324 (possibly the total number of bins),
# and multiply by 100 to get the percentage of coverage.
data_comparison["Unplanned (%)"] = [
    len(np.unique(stk_object.Holding_Data[stk_object.Holding_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 
    for tar_num in range(len(stk_object.targets))]


# Calculate the percentage of unique planned bin numbers for each target
# 'stk_object.Planned_Data' contains planned data, filtered for each target ('Target' == tar_num)
# Similar to the 'Unplanned (%)', it counts the unique 'Bin Number' values and calculates the percentage.
data_comparison["Planned (%)"] = [
    len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 
    for tar_num in range(len(stk_object.targets))]


# Calculate the maximum planned time for each target (in days)
# 'stk_object.Planned_Data' is filtered for each target, and the maximum 'Time' value is taken.
# 'Time' is assumed to be in seconds, so it is divided by 86400 (seconds in a day) to convert to days.
data_comparison["Planned (Time)"] = [
    max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 
    for tar_num in range(len(stk_object.targets))]


# Convert the dictionary into a pandas DataFrame for easier analysis and visualization
data_comparison = pd.DataFrame(data_comparison)


# Print the DataFrame to display the data for each target
print(data_comparison)


# Print summary statistics (like mean, standard deviation, min, max, etc.) for each column of the DataFrame
print(data_comparison.describe())

   Unplanned (%)  Planned (%)  Planned (Time)
0      44.753086    19.444444        0.954959
1      52.160494    14.506173        0.886985
2      47.530864     5.555556        0.882656
3      51.234568    13.888889        0.950116
4      37.962963    10.493827        0.827454
5      48.148148    13.580247        0.945290
6      42.901235     7.407407        0.822778
7      40.123457     8.333333        0.891111
8      48.148148    13.580247        0.816869
9      51.543210    14.197531        0.948872
       Unplanned (%)  Planned (%)  Planned (Time)
count      10.000000    10.000000       10.000000
mean       46.450617    12.098765        0.892709
std         4.897919     4.127554        0.055903
min        37.962963     5.555556        0.816869
25%        43.364198     8.873457        0.841254
50%        47.839506    13.580247        0.889048
75%        50.462963    14.120370        0.947976
max        52.160494    19.444444        0.954959


In [11]:
# import plotly.io as pio
# pio.renderers.default = 'notebook'
# %pip install nbformat

# Create a scatter plot using Plotly Express
# 'stk_object.Planned_Data' is sorted by the 'Target' column for proper animation and visualization order
# 'x' axis represents 'Time', and 'y' axis represents 'Bin Number'
# 'hover_data' shows 'Bin Number' when hovering over the points on the plot
# 'color' differentiates points based on the 'Satellite' column, using different colors for each satellite
# 'animation_frame' allows the plot to animate, with each frame representing a different 'Target'
fig = px.scatter(
    stk_object.Planned_Data.sort_values(by="Target"),
    x="Time",
    y="Bin Number",
    hover_data="Bin Number",
    color="Satellite",
    animation_frame="Target"
)

# Display the generated scatter plot with animation
fig.show()